In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from sklearn.cluster import KMeans
import tqdm

Achando o melhor valor de K usando o método do cotovelo.

In [ ]:
# Lê a imagem do disco
img = cv2.imread('path')

# Converte a imagem de BGR para RGB
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Vectoriza a imagem, convertendo-a em uma matriz 2D de pixels
vectorized_img = img.reshape((-1,3))
vectorized_img = np.float32(vectorized_img)

# Define uma lista de números de clusters para testar
n_clusters = list(range(4, 15 + 1, 1))

# Inicializa listas para armazenar os modelos KMeans e suas respectivas inércias
kmeans = []
inertias = []

# Itera sobre os diferentes números de clusters
for i in tqdm.trange(len(n_clusters)):
    # Inicializa um modelo KMeans com o número de clusters atual
    kmeans.append(KMeans(n_clusters = n_clusters[i], random_state = 42))

    # Ajusta o modelo aos dados vectorizados da imagem
    kmeans[-1].fit(vectorized_img)

    # Calcula e armazena a inércia do modelo atual
    inertias.append(kmeans[-1].inertia_)

# Plota os resultados
plt.figure(figsize = [20, 5])

# Subplot 1: Gráfico de Inércia em função do número de clusters
plt.subplot(1, 2, 1)
plt.plot(n_clusters, inertias, "-o")
plt.xlabel("$k$", fontsize = 14)
plt.ylabel("Inertia", fontsize = 14)
plt.grid(True)

# Subplot 2: Gráfico da mudança na inércia em função do número de clusters
plt.subplot(1, 2, 2)
plt.plot(n_clusters[:-1], np.diff(inertias), "-o")
plt.xlabel("$k$", fontsize = 14)
plt.ylabel("Change in inertia", fontsize = 14)
plt.grid(True)

# Mostra os gráficos
plt.show()

Dado que a diferença de Ks chega em um platô no K = 10, esse será o valor escolhido para essa parte do pré processamento

In [ ]:
def kmeansImagem(caminho_imagem, k, attempts=10):
    img = cv2.imread(caminho_imagem)

    # Redimensionar a imagem para um vetor 1D
    vectorized_img = img.reshape((-1,3))
    vectorized_img = np.float32(vectorized_img)

    # Definir critérios para o algoritmo K-means
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)

    # Executar o algoritmo K-means
    ret, label, center = cv2.kmeans(vectorized_img, k, None, criteria, attempts, cv2.KMEANS_PP_CENTERS)
    center = np.uint8(center)

    # Recriar a imagem a partir dos clusters encontrados
    res = center[label.flatten()]
    result_image = res.reshape((img.shape))

    return result_image

def sharpenImage(imagem):
    # Cria um kernel para afiar a imagem
    kernel = np.array([[-1,-1,-1],
                      [-1,9,-1],
                      [-1,-1,-1]])

    # Aplica o kernel na imagem usando filter2D
    sharpened_image = cv2.filter2D(imagem, -1, kernel)

    return sharpened_image


def processar_todas_imagens(diretorio, k, attempts=10):
    # Lista todas as imagens no diretório
    imagens = os.listdir(diretorio)
    imagensRetorno = []

    # Itera sobre todas as imagens no diretório
    for imagem_nome in imagens:
        caminho_imagem = os.path.join(diretorio, imagem_nome)

        # Aplica o algoritmo de k-means na imagem
        kmeansImage = kmeansImagem(caminho_imagem, k, attempts)

        # Afia a imagem resultante
        sharpenedImage = sharpenImage(kmeansImage)

        # Adiciona a imagem afiada à lista de retorno
        imagensRetorno.append(sharpenedImage)
    return imagensRetorno

In [ ]:
# Define o caminho da imagem
caminho_imagem = '/content/drive/MyDrive/Agro-data/dataset_inteli/dataset_inteli/tci_pngs/1401_2020-8-28_S2L1C_21JYN_TCI.png'

# Lê a imagem do disco
imagemTeste = cv2.imread(caminho_imagem)

# Plota a imagem
plt.figure(figsize=(20, 15))  # Define o tamanho da figura
imagemTeste = cv2.cvtColor(imagemTeste, cv2.COLOR_BGR2RGB)  # Converte a imagem de BGR para RGB
plt.title('Original Image')   # Define o título do gráfico como "Original Image"
plt.imshow(imagemTeste)       # Exibe a imagem
plt.show()                    # Mostra a figura

In [ ]:
# Define o tamanho da figura
plt.figure(figsize=(20, 15))

# Aplica o algoritmo de k-means na imagem
imagemTesteKmeans = kmeansImagem(caminho_imagem, 3, 10)

# Converte a imagem de BGR para RGB
imagemTesteKmeans = cv2.cvtColor(imagemTesteKmeans, cv2.COLOR_BGR2RGB)

# Define o título do gráfico como "Kmeans Image"
plt.title('Kmeans Image')

# Exibe a imagem
plt.imshow(imagemTesteKmeans)

# Mostra a figura
plt.show()

In [ ]:
# Lê a imagem do disco
img = cv2.imread(caminho_imagem)

# Aplica a função de afiação à imagem
img = sharpenImage(img)

# Converte a imagem de BGR para RGB
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Define o tamanho da figura
plt.figure(figsize=(20, 15))

# Define o título do gráfico como "Sharpen Image"
plt.title('Sharpen Image')

# Exibe a imagem
plt.imshow(img)

# Mostra a figura
plt.show()

In [ ]:
# Define o diretório contendo as imagens a serem processadas
diretorio = 'drive/MyDrive/Agro-data/dataset_inteli/dataset_inteli/tci_pngs/'

# Processa todas as imagens no diretório usando a função processar_todas_imagens
imagens = processar_todas_imagens(diretorio, 10)

# Itera sobre cada imagem processada
for imagem in imagens:
    # Converte a imagem de BGR para RGB
    img = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)

    # Define o tamanho da figura
    plt.figure(figsize=(20, 15))

    # Exibe a imagem
    plt.imshow(img)

    # Mostra a figura
    plt.show()